In [1]:
import numpy as np
from data import get_data
from model import *
from model_constraints import *

In [2]:
def get_variables(data):
    employee_dv =  {(m, d, t): 0
                for m in data["STAFF"]
                for d in data["DAYS"]
                for t in data["SHIFTS"]}

    above_dv = {(d, t, k): 0
                for d in data["DAYS"]
                for t in data["SHIFTS"]
                for k in data["SCENARIOS"]}

    below_dv = {(d, t, k): 0
                for d in data["DAYS"]
                for t in data["SHIFTS"]
                for k in data["SCENARIOS"]}
    return employee_dv, above_dv, below_dv

In [3]:
def get_results(data, file, employee_dv, above_dv, below_dv):
    with open(file) as soltext:
        temp = [0]  * len(data["DAYS"])
        count = []
        for _ in range(len(data["SHIFTS"])):
            count.append(temp.copy())
        soltext.readline()
        soltext.readline()
        soltext.readline()
        for d in data["DAYS"]:
            assert(soltext.readline() == f"Day: {d}\n")

            while True:
                line = soltext.readline()
                if line == '\n':
                    break
                else:
                    m, t = line.strip().split(" works shift ")
                    employee_dv[m,d,t] = 1
                    count[data["SHIFTS"].index(t)][d - 1] += 1

    for d in data["DAYS"]:
        for t in data["SHIFTS"]:
            for k in data["SCENARIOS"]:
                requirement = [item for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
                coverage_req = requirement[2]
                if count[data["SHIFTS"].index(t)][d - 1] > coverage_req:
                    above_dv[d, t, k] = count[data["SHIFTS"].index(t)][d - 1] - coverage_req
                elif count[data["SHIFTS"].index(t)][d - 1] < coverage_req:
                    below_dv[d, t, k] = coverage_req - count[data["SHIFTS"].index(t)][d - 1]

    return employee_dv, above_dv, below_dv

In [4]:
def get_cost(data, employee_dv, above_dv, below_dv, scenarios): 
    return sum(data["SECTION_SHIFT_OFF_REQUESTS"].get((s, d - 1, t), 0) * employee_dv[(s, d, t)]
            for s in data["STAFF"]
            for d in data["DAYS"]
            for t in data["SHIFTS"]) \
        +\
        sum(data["SECTION_SHIFT_ON_REQUESTS"].get((s, d - 1, t), 0) * (1 - employee_dv[(s, d, t)])
            for s in data["STAFF"]
            for d in data["DAYS"]
            for t in data["SHIFTS"])\
        +\
        sum([
        #overstaffing penalty
        (1 / len(scenarios)) * sum(above_dv[d, t, k] * [item[4] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in scenarios)\
        + \
        #understaffing penalty
        (1 / len(scenarios)) * sum(below_dv[d, t, k] * [item[3] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
            for d in data["DAYS"]
            for t in data["SHIFTS"]
            for k in scenarios)
        ]
        )

In [5]:
def get_average_solution_cost(data, file, scenarios):
    employee_dv, above_dv, below_dv = get_variables(data)
    employee_dv, above_dv, below_dv = get_results(data, file, employee_dv, above_dv, below_dv)
    cost = 0
    for scenario in scenarios:
        cost += get_cost(data, employee_dv, above_dv, below_dv, scenario)
    print(f"Average solution cost {cost / len(scenarios)}")

In [14]:
scenarios1 = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],]
scenarios2 = [[2], [7], [6], [1], ]
scenarios3 = [[4], [3], [8], [9]]
instances = [0,1,2,3,4,5,6,7,8,9,'ea']
instances2 = [0,1,2,3,4,5,6,7,8,9]

In [15]:
index = 1
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")

for instance in instances:
    get_average_solution_cost(data, f"output/instance1/instance1s_{instance}.txt", scenarios1)
print()


for instance in instances:
    get_average_solution_cost(data, f"output/instance1/instance1s_{instance}.txt", scenarios2)
print()


for instance in instances:
    get_average_solution_cost(data, f"output/instance1/instance1s_{instance}.txt", scenarios3)
print()


Average solution cost 1327.871847222534
Average solution cost 1357.9592059194538
Average solution cost 1555.3427444362565
Average solution cost 1590.5914498498055
Average solution cost 1960.3116592141719
Average solution cost 1372.5830273706827
Average solution cost 1321.8839984053857
Average solution cost 1401.2734078681601
Average solution cost 1710.926646217737
Average solution cost 2095.6688554346465
Average solution cost 1179.393994331554

Average solution cost 2683.934802260071
Average solution cost 2500.2203802875015
Average solution cost 2623.1207223843403
Average solution cost 3347.30265533715
Average solution cost 3686.3484973550094
Average solution cost 2795.5917340520264
Average solution cost 2422.1828471058047
Average solution cost 2504.682210673938
Average solution cost 3555.5582877555203
Average solution cost 3827.0962624255726
Average solution cost 2379.9574886667115

Average solution cost 360.2029673955879
Average solution cost 484.71899967494244
Average solution cost 

In [8]:
index = 2
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")

for instance in instances:
    get_average_solution_cost(data, f"output/instance2/instance2s_{instance}.txt", scenarios1)
print()

Average solution cost 2379.8016254103586
Average solution cost 2784.971639740859
Average solution cost 3208.5679762380155
Average solution cost 3051.0606188678794
Average solution cost 3665.0464731654383
Average solution cost 2401.2145340904217
Average solution cost 2970.143096290537
Average solution cost 3661.910017127818
Average solution cost 3187.2955441488043
Average solution cost 4134.985816435423
Average solution cost 2183.0159192091387



In [9]:
index = 3
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")


for instance in instances:
    get_average_solution_cost(data, f"output/instance3/instance3s_{instance}.txt", scenarios1)
print()

Average solution cost 3230.68318556206
Average solution cost 3670.3535852846435
Average solution cost 3687.66647582141
Average solution cost 4620.475419767899
Average solution cost 5009.450362364334
Average solution cost 3333.0006287679694
Average solution cost 3963.211226161344
Average solution cost 5211.978087929407
Average solution cost 4857.979565537057
Average solution cost 6462.576798252653
Average solution cost 2984.681423245165



In [10]:
index = 4
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")

for instance in instances:
    get_average_solution_cost(data, f"output/instance4/instance4s_{instance}.txt", scenarios1)
print()

Average solution cost 4711.620289283766
Average solution cost 4624.460521533454
Average solution cost 5504.05323767518
Average solution cost 5321.963617743726
Average solution cost 5710.439745466977
Average solution cost 4926.265519431688
Average solution cost 4678.354615980692
Average solution cost 6043.465245737725
Average solution cost 5827.202031338783
Average solution cost 7308.706903722135
Average solution cost 4184.423321620143



In [11]:
index = 5
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")

for instance in instances:
    get_average_solution_cost(data, f"output/instance5/instance5s_{instance}.txt", scenarios1)
print()

Average solution cost 4129.5588451354215
Average solution cost 4152.890898870181
Average solution cost 4665.80544825657
Average solution cost 5506.913918132299
Average solution cost 6502.1540866320665
Average solution cost 4382.52812610663


Average solution cost 5064.503168203201
Average solution cost 6263.992953225523
Average solution cost 5835.123857360582
Average solution cost 7696.98065539158
Average solution cost 3718.2867782604567



In [16]:
index = 6
np.random.seed(index)
data = get_data(f"output/instance{index}/instance{index}.txt")

for instance in instances:
    get_average_solution_cost(data, f"output/instance6/instance6s_{instance}.txt", scenarios1)
print()

Average solution cost 6796.579261689457
Average solution cost 6829.756147891108
Average solution cost 7981.789849639562
Average solution cost 8139.8127828638635
Average solution cost 9247.335386588516
Average solution cost 6626.055284810672
Average solution cost 7582.469598947576
Average solution cost 9050.898398246405
Average solution cost 8499.153709799637
Average solution cost 11671.169338006042
Average solution cost 5955.274823297788



FA

In [20]:
for index in instances2:
    cost = 0
    for instance in instances2:
        with open(f"output/instance1/recourse/instance1_{index}/instance1s_{instance}fa.txt") as file:
            file.readline()
            cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
    print(cost / 10)


cost = 0
for instance in instances2:
    with open(f"output/instance1/recourse/instance1_fa/instance1s_{instance}fa.txt") as file:
        file.readline()
        cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
print(cost / 10)

2257.5
1283.4
1478.1
1509.6
1862.7
1297.2
1262.4
1336.0
1627.9
2001.2
1119.1


In [21]:
for index in instances2:
    cost = 0
    for instance in instances2:
        with open(f"output/instance2/recourse/instance2_{index}/instance2s_{instance}fa.txt") as file:
            file.readline()
            cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
    print(cost / 10)


cost = 0
for instance in instances2:
    with open(f"output/instance2/recourse/instance2_fa/instance2s_{instance}fa.txt") as file:
        file.readline()
        cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
print(cost / 10)

2306.7
2695.7
3111.9
2950.5
3550.0
2311.3
2899.5
3550.1
3080.3
3996.9
2127.3


In [22]:
for index in instances2:
    cost = 0
    for instance in instances2:
        with open(f"output/instance3/recourse/instance3_{index}/instance3s_{instance}fa.txt") as file:
            file.readline()
            cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
    print(cost / 10)


cost = 0
for instance in instances2:
    with open(f"output/instance3/recourse/instance3_fa/instance3s_{instance}fa.txt") as file:
        file.readline()
        cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
print(cost / 10)

3098.1
3529.2
3570.4
4433.9
4817.1
3191.4
3872.6
5058.4
4664.0
6215.1
2863.1


In [ ]:
for index in instances2:
    cost = 0
    for instance in instances2:
        with open(f"output/instance4/recourse/instance4_{index}/instance4s_{instance}fa.txt") as file:
            file.readline()
            cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
    print(cost / 10)


cost = 0
for instance in instances2:
    with open(f"output/instance4/recourse/instance4_fa/instance4s_{instance}fa.txt") as file:
        file.readline()
        cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
print(cost / 10)

4476.2
4379.8
5222.3
5078.5
5460.2
4686.7
4493.2
5816.8
5555.3
6983.9
3958.8


In [23]:
for index in instances2:
    cost = 0
    for instance in instances2:
        with open(f"output/instance5/recourse/instance5_{index}/instance5s_{instance}fa.txt") as file:
            file.readline()
            cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
    print(cost / 10)


cost = 0
for instance in instances2:
    with open(f"output/instance5/recourse/instance5_fa/instance5s_{instance}fa.txt") as file:
        file.readline()
        cost += int(file.readline().strip().split("Maximum of objective function: ")[1])
print(cost / 10)

3982.8
4018.9
4493.4
5303.2
6276.6
4207.5
4914.0
6108.2
5621.6
7394.7
3630.1
